Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1273]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import heapq
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree



In [1274]:
PUZZLE_DIM =4
action = namedtuple('Action', ['pos1', 'pos2'])

In [1275]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state


In [1276]:
RANDOMIZE_STEPS = 1_000
target_state=np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
start_state = target_state
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    start_state = do_action(start_state, choice(available_actions(start_state)))

# start_state = np.array([np.array([0,2,1]),np.array([3,7,5]),np.array([8,6,4])])

Randomizing:   0%|          | 0/1000 [00:00<?, ?it/s]

In [1277]:
#both x and y of the target matrix will be the same so we compute them only once. They are used to compute manhattan distance.
x_target=np.zeros(PUZZLE_DIM**2)
y_target=np.zeros(PUZZLE_DIM**2)
for i in range(1,PUZZLE_DIM**2):
            x_target[i],y_target[i]=np.where(target_state==i)

class Position: #The position class represent the state of the game with the f (h+g) embedded.
    def __init__(self,grid:np.array,g:int,prev_pos=None) -> None:
        self.grid=grid
        self.prev_pos=prev_pos
        self.g=g
        self.f=int(self.__h()+g)
    def __h(self):
        #This is the heuristic: MANHATTAN DISTANCE over every element
        tot_dist=0
        for i in range(1,PUZZLE_DIM**2):
            # x_1,y_1=np.where(target_state==i) 
            x_2,y_2=np.where(self.grid==i)
            tot_dist+=abs(x_target[i]-x_2)+abs(y_target[i]-y_2)
        return tot_dist
    

    
    #For heap
    def __lt__(self, other):
     return self.f<other.f
    

    def __hash__(self):
            return hash(self.grid.tobytes())
    
    def __eq__(self, other):
        return np.array_equal(self.grid, other.grid) 






C:\Users\Ferra\AppData\Local\Temp\ipykernel_18044\2787315892.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_target[i],y_target[i]=np.where(target_state==i)


In [1278]:

def a_star(start_pos:Position):

    curr_pos=start_pos
    closed_set = set() #Visited Positions
    priority_heap=[] #Priority queue of open_set elements
    # open_set={hash(start_pos) : start_pos.g} 

    while True :
        # open_set.pop(hash(curr_pos))
        closed_set.add(hash(curr_pos)) 
        # if((curr_pos.grid==target_state).all()):
        if(curr_pos.f-curr_pos.g==0):
            print("Elems open list: ", len(priority_heap))
            print("Evaluated pos (cost): ",len(closed_set)-1) #-1 because of starting position
            return curr_pos
  
        new_actions=available_actions(curr_pos.grid)
        for action in new_actions:
            new_pos=Position(do_action(curr_pos.grid,action),curr_pos.g+1,curr_pos)
            if hash(new_pos) in closed_set:
                continue
            # elif hash(new_pos) not in open_set or new_pos.g < open_set[hash(new_pos) ]: 
                # open_set[hash(new_pos) ] = new_pos.g
            heapq.heappush(priority_heap, new_pos) 
           
            
        while hash(curr_pos) in closed_set: #if curr_pos is not in closed_set means that we found a way to reach the same grid with lower g, so we just pop it and look for another Position
      
            curr_pos=heapq.heappop(priority_heap) 


        # if(evaluated_pos%50000==0):
        #     ic(curr_pos.grid)

start_pos=Position(start_state,0)


print("Start grid:\n",start_pos.grid,"\nStart heuristic: ",start_pos.f)

found_sol=a_star(start_pos)   


print("Steps: ",found_sol.g)
print("Found solution (backward): ")
while True:
    print(found_sol.grid,"\n ")
    if(found_sol.prev_pos is None):
        break
    found_sol=found_sol.prev_pos




C:\Users\Ferra\AppData\Local\Temp\ipykernel_18044\2787315892.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.f=int(self.__h()+g)


Start grid:
 [[ 6 10  1  9]
 [11  2 12  8]
 [ 5  4  0  3]
 [13 14  7 15]] 
Start heuristic:  28
Elems open list:  5915
Evaluated pos (cost):  5709
Steps:  36
Found solution (backward): 
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]] 
 
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14  0 15]] 
 
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13  0 14 15]] 
 
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9  0 11 12]
 [13 10 14 15]] 
 
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 0  9 11 12]
 [13 10 14 15]] 
 
[[ 1  2  3  4]
 [ 0  6  7  8]
 [ 5  9 11 12]
 [13 10 14 15]] 
 
[[ 1  2  3  4]
 [ 6  0  7  8]
 [ 5  9 11 12]
 [13 10 14 15]] 
 
[[ 1  2  3  4]
 [ 6  9  7  8]
 [ 5  0 11 12]
 [13 10 14 15]] 
 
[[ 1  2  3  4]
 [ 6  9  7  8]
 [ 5 11  0 12]
 [13 10 14 15]] 
 
[[ 1  2  3  4]
 [ 6  9  0  8]
 [ 5 11  7 12]
 [13 10 14 15]] 
 
[[ 1  2  0  4]
 [ 6  9  3  8]
 [ 5 11  7 12]
 [13 10 14 15]] 
 
[[ 1  2  4  0]
 [ 6  9  3  8]
 [ 5 11  7 12]
 [13 10 14 15]] 
 
[[ 1  2  4  8]
 [ 6  9  3  0]
 [ 5 11  7 12]
 